In [ ]:
import mlflow
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import tensorflow_addons as tfa

from tensorflow.keras.callbacks import EarlyStopping

from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset

import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")

## Load Data

In [ ]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

## Data Processing

In [6]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

## Baseline

In [10]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20)
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

## CatBoost

In [11]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL]) 
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL]) 
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [ ]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")
    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

## MLP

In [114]:
def build_mlp(params):
    mlp = Sequential([
        Dense(params["layer1_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer2_size"], activation=params['activation']),
        Dropout(params['dropout_rate']),
        Dense(params["layer3_size"], activation=params['activation']),
        Dense(1, activation='relu')
    ])
    return mlp

def train_mlp(mlp, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )
    mlp.compile(
        optimizer=optimizer,
        loss=MeanSquaredError(name="mse"),
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
    )

    early = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = mlp.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return mlp


def mlp_mlflow_run(
    name,
    mlp_params,
    train_params,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.log_params(mlp_params)
        mlflow.log_params(train_params)
        mlflow.set_tag("model_name", "MLP")
        mlp = build_mlp(mlp_params)
        mlp = train_mlp(mlp, train_params, train_dataset, val_dataset)
        test_preds = mlp.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds.ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(mlp, "tf_models")

In [ ]:
# To TF Dataset
mlp_train_ds = tf.data.Dataset.from_tensor_slices((X_train[FEATURES], X_train[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_val_ds = tf.data.Dataset.from_tensor_slices((X_val[FEATURES], X_val[LABEL])).batch(512).shuffle(512*4).prefetch(512)
mlp_test_ds = tf.data.Dataset.from_tensor_slices(test_data[FEATURES]).batch(512).prefetch(512)

mlp_params = {
    "layer1_size": 512,
    "layer2_size": 128,
    "layer3_size": 64,
    "dropout_rate": 0.3,
    "activation": 'relu'

}
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

mlp_mlflow_run(
    "mlp_base",
    mlp_params,
    train_params,
    mlp_train_ds,
    mlp_val_ds,
    mlp_test_ds,
    test_data[LABEL],
)

In [ ]:
mlp_mlflow_run

## FT Transformers

In [ ]:
# To TF Dataset
train_dataset = df_to_dataset(X_train[FEATURES + [LABEL]], LABEL, shuffle=True)
val_dataset = df_to_dataset(X_val[FEATURES + [LABEL]], LABEL, shuffle=False)  # No shuffle
test_dataset = df_to_dataset(test_data[FEATURES], shuffle=False) # No target, no shuffle

In [97]:
def build_fttransformer(
    params_to_log, params_to_skip, out_dim=1, out_activation="relu"
):
    # Define encoder
    ft_encoder = FTTransformerEncoder(
        **params_to_log,
        **params_to_skip
    )
    # Add prediction head to the encoder
    ft_transformer = FTTransformer(
        encoder=ft_encoder,
        out_dim=out_dim,
        out_activation=out_activation,
    )

    return ft_transformer


def train_model(model, train_params, train_dataset, val_dataset):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=train_params["learning_rate"],
        weight_decay=train_params["weight_decay"],
    )

    model.compile(
        optimizer=optimizer,
        loss={
            "output": tf.keras.losses.MeanSquaredError(name="mse"),
            "importances": None,
        },
        metrics={
            "output": [tf.keras.metrics.RootMeanSquaredError(name="rmse")],
            "importances": None,
        },
    )

    early = EarlyStopping(
        monitor="val_output_loss",
        mode="min",
        patience=train_params["early_stop_patience"],
        restore_best_weights=True,
    )
    callback_list = [early]

    hist = model.fit(
        train_dataset,
        epochs=train_params["num_epochs"],
        validation_data=val_dataset,
        callbacks=callback_list,
    )
    return model


In [103]:
mlflow.tensorflow.autolog(disable=True)


def fttransformer_mlflow_run(
    name,
    encoder_params,
    train_params,
    params_to_skip,
    train_dataset,
    val_dataset,
    test_dataset,
    y_test,
):
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", "FTTransformer")
        # Log the params
        mlflow.log_params(encoder_params)
        mlflow.log_params(train_params)
        # Build and train
        ft_transformer = build_fttransformer(
            encoder_params,
            params_to_skip,
            out_dim=1,
            out_activation="relu",
        )
        ft_transformer = train_model(
            ft_transformer, train_params, train_dataset, val_dataset
        )
        # Evaluate
        test_preds = ft_transformer.predict(test_dataset)
        test_rms = mean_squared_error(
            y_test, test_preds["output"].ravel(), squared=False
        )
        mlflow.log_metric("test_rmse", test_rms)
        mlflow.tensorflow.log_model(ft_transformer, "tf_models")


In [104]:
train_params = dict(
    learning_rate=0.001, weight_decay=0.00001, early_stop_patience=10, num_epochs=1000
)

params_to_skip = dict(
    numerical_data=X_train[NUMERIC_FEATURES].values,
    categorical_data=None,
    y=X_train[LABEL].values
)

### Linear Embeddings

In [ ]:
linear_embeddings_params = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type="linear",
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='linear',
    encoder_params=linear_embeddings_params,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

### Periodic

In [ ]:
periodic_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='periodic',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

fttransformer_mlflow_run(
    name='periodic',
    encoder_params=periodic_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

### PLE - Quantile Binning

In [ ]:
pleq_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
)

pleq_params_to_skip = params_to_skip.copy()
pleq_params_to_skip['y'] = None

fttransformer_mlflow_run(
    name='ple_quantile',
    encoder_params=pleq_params_to_log,
    train_params=train_params,
    params_to_skip=pleq_params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

### PLE - Target Binning

In [ ]:
plet_params_to_log = dict(
    numerical_features=NUMERIC_FEATURES,
    categorical_features=[],
    numerical_embedding_type='ple',
    numerical_bins=128,
    embedding_dim=64,
    depth=3,
    heads=6,
    attn_dropout=0.3,
    ff_dropout=0.3,
    explainable=True,
    task='regression',
    ple_tree_params = {
        "min_samples_leaf": 20,
    }
)


fttransformer_mlflow_run(
    name='ple_target',
    encoder_params=plet_params_to_log,
    train_params=train_params,
    params_to_skip=params_to_skip,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    y_test=test_data[LABEL],
)

In [87]:
model_id = "" # take it from mlflow
loaded_ft = mlflow.tensorflow.load_model(model_id)